In [64]:
study = "1.2.392.200036.9116.2.5.1.48.1216060901.1336345628.42"
series = "1.2.392.200036.9116.2.1216060901.1336346550.3.1004100001.7"
instance = "1.2.392.200036.9116.2.1216060901.1336346841.684518.7.1277"
r, url = query_vna(user='wangkcc4', pw='', filename="test.dcm", study=study, series=series, instance=instance)

In [ ]:
#http://server:port/ACUORest/dicomrs/search/studies?PatientID=212121&includefield=all
#http://server:port/ACUORest/dicomrs/search/studies?00100010=A*&limit=1
#http://server:port/ACUORest/dicomrs/search/studies/1.2.840.113619.2.30.1.1762288588.1465.911090938.258/series

In [91]:
r, url = search_vna(user='wangkcc4', pw='', region="prod", patientID="2")

In [90]:
def search_vna(user, pw, patientID=None, region='test'):

    if region == 'test':
        host = 'vnatest1vt'
        port = '8083'
    elif region == 'prod':
        host = '10.47.11.220'
        port = '8083'
    else:
        raise ValueError("Unsupported region")


    url = ''.join(['http://', host, ':', port,
                   "/AcuoREST/dicomrs/search/studies?PatientID=",
                    patientID,"*"])

    url += "&limit=2"
    #url += "&includefield=all"

    r = requests.get(url, auth=(user, pw)) #, headers=headers
    #if r.status_code != 200:
        #raise ValueError("Invalid request (response code %d) for URL: %s" % (r.status_code, url))
        
    return r, url

In [95]:
r.status_code

403

In [93]:
r.headers

{'Server': 'Microsoft-HTTPAPI/2.0', 'Content-Length': '0', 'Date': 'Tue, 26 Sep 2017 18:25:35 GMT'}

In [94]:
url

'http://10.47.11.220:8083/AcuoREST/dicomrs/search/studies?PatientID=2*&limit=2'

In [ ]:
r.json()

In [63]:
import requests

def query_vna(user, pw, filename, study=None, series=None, instance=None, region='test', metadata=False):

    if region == 'test':
        host = 'vnatest1vt'
        port = '8083'
    elif region == 'prod':
        host = 'VNAASX1VP'
        port = 'TBD'
    else:
        raise ValueError("Unsupported region")


    if metadata:
        url = ''.join(['http://', host, ':', port,
                       "/AcuoREST/dicomrs/retrieve/studies/",
                        study])

        if series is not None:
            url += "/series/" + series
            if instance is not None:
                url += "/instances/" + instance

        url += "/metadata"

    else:
        url = ''.join(['http://', host, ':', port,
                       "/AcuoREST/wadoget?requestType=WADO&contentType=application/dicom&studyUID=",
                        study])

        if series is not None:
            url += "&seriesUID=" + series
            if instance is not None:
                url += "&objectUID=" + instance
                
        url += "&aonymize"

    r = requests.get(url, auth=(user, pw)) #, headers=headers

    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
            
    #if r.status_code != 200:
        #raise ValueError("Invalid request (response code %d) for URL: %s" % (r.status_code, url))
        
    return r, url

In [56]:
url

'http://vnatest1vt:8083/AcuoREST/wadoget?requestType=WADO&contentType=application/dicom&studyUID=1.2.392.200036.9116.2.5.1.48.1216060901.1336345628.42&seriesUID=1.2.392.200036.9116.2.1216060901.1336346550.3.1004100001.7&objectUID=1.2.392.200036.9116.2.1216060901.1336346841.684518.7.1277&anonymize=yes'

In [67]:
r.content

b'<string xmlns="http://schemas.microsoft.com/2003/10/Serialization/">Wado Exception - parameter frame not supported for contentType application/dicom</string>'

In [65]:
r.headers

{'Content-Type': 'application/xml; charset=utf-8', 'Server': 'Microsoft-HTTPAPI/2.0', 'Date': 'Tue, 26 Sep 2017 14:36:29 GMT', 'Transfer-Encoding': 'chunked'}

In [66]:
r.status_code

501